In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score

In [13]:
# Load and preprocess data
file_path = 'log2.csv'
data = pd.read_csv(file_path)

# Encode categorical column 'Action'
label_encoder = LabelEncoder()
data['Action'] = label_encoder.fit_transform(data['Action'])

# Separate features and target
X = data.drop(columns=['Action'])
y = data['Action']

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Move tensors to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X_train_tensor = X_train_tensor.to(device)
X_test_tensor = X_test_tensor.to(device)
y_train_tensor = y_train_tensor.to(device)
y_test_tensor = y_test_tensor.to(device)

if torch.cuda.is_available():
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("Using CPU")

Using GPU: NVIDIA GeForce RTX 2070 SUPER


In [15]:
# Define RNN model
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, pooling_type, num_layers=1):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers=num_layers, batch_first=True)
        self.pooling_type = pooling_type
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out, _ = self.rnn(x)
        if self.pooling_type == 'max':
            out = torch.max(out, dim=1).values
        elif self.pooling_type == 'avg':
            out = torch.mean(out, dim=1)
        out = self.fc(out)
        return out

# Hyperparameters and configurations
input_size = X_train_tensor.shape[1]
num_classes = len(label_encoder.classes_)
hidden_sizes = [32, 64, 128]
pooling_types = ['max', 'avg']
epochs_list = [5, 50, 100, 250, 350]
optimizers = ['SGD', 'RMSprop', 'Adam']

In [16]:
# Results logging
results = []

# Training loop with hyperparameter tuning
best_model = None
best_accuracy = 0
best_params = {}

for hidden_size in hidden_sizes:
    for pooling_type in pooling_types:
        for optimizer_name in optimizers:
            for epochs in epochs_list:
                print(f"--- Hidden Size: {hidden_size}, Pooling: {pooling_type}, Optimizer: {optimizer_name}, Epochs: {epochs} ---")
                model = RNNModel(input_size, hidden_size, num_classes, pooling_type, num_layers=2).to(device)
                criterion = nn.CrossEntropyLoss()

                if optimizer_name == 'SGD':
                    optimizer = optim.SGD(model.parameters(), lr=0.01)
                elif optimizer_name == 'RMSprop':
                    optimizer = optim.RMSprop(model.parameters(), lr=0.01)
                elif optimizer_name == 'Adam':
                    optimizer = optim.Adam(model.parameters(), lr=0.01)

                # Early stopping setup
                early_stop_patience = 10
                best_loss = float('inf')
                patience_counter = 0

                # Train model
                for epoch in range(epochs):
                    model.train()
                    optimizer.zero_grad()
                    outputs = model(X_train_tensor.unsqueeze(1))
                    loss = criterion(outputs, y_train_tensor)
                    loss.backward()
                    optimizer.step()

                    # Validate model
                    model.eval()
                    with torch.no_grad():
                        val_outputs = model(X_test_tensor.unsqueeze(1))
                        val_loss = criterion(val_outputs, y_test_tensor).item()
                        val_predictions = val_outputs.argmax(dim=1)
                        val_accuracy = accuracy_score(y_test_tensor.cpu(), val_predictions.cpu()) * 100

                    # Log epoch details
                    print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item():.4f}, Val Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.2f}%")

                    # Early stopping check
                    if val_loss < best_loss:
                        best_loss = val_loss
                        patience_counter = 0
                    else:
                        patience_counter += 1

                    if patience_counter >= early_stop_patience:
                        print("Early stopping triggered.")
                        break

                # Evaluate model
                model.eval()
                with torch.no_grad():
                    predictions = model(X_test_tensor.unsqueeze(1)).argmax(dim=1)
                    accuracy = accuracy_score(y_test_tensor.cpu(), predictions.cpu())

                # Log results
                # Log results
                results.append({
                    'hidden_size': hidden_size,
                    'pooling_type': pooling_type,
                    'optimizer': optimizer_name,
                    'epochs': epochs,
                    'best_loss': best_loss,  # Menyimpan best_loss
                    'accuracy': accuracy     # Menyimpan akurasi akhir
                })


                # Check for best model
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_model = model
                    best_params = {
                        'hidden_size': hidden_size,
                        'pooling_type': pooling_type,
                        'optimizer': optimizer_name,
                        'epochs': epochs
                    }

# Save all experiment results to CSV
results_df = pd.DataFrame(results)
results_df.to_csv('hyperparameter_tuning_results.csv', index=False)
print("Hasil semua eksperimen disimpan ke 'hyperparameter_tuning_results.csv'")
print("Best Hyperparameters:", best_params)
print("Best Accuracy:", best_accuracy)

--- Hidden Size: 32, Pooling: max, Optimizer: SGD, Epochs: 5 ---
Epoch 1/5, Loss: 1.2435, Val Loss: 1.2388, Accuracy: 56.95%
Epoch 2/5, Loss: 1.2390, Val Loss: 1.2343, Accuracy: 57.04%
Epoch 3/5, Loss: 1.2345, Val Loss: 1.2298, Accuracy: 57.09%
Epoch 4/5, Loss: 1.2301, Val Loss: 1.2254, Accuracy: 57.14%
Epoch 5/5, Loss: 1.2257, Val Loss: 1.2211, Accuracy: 57.16%
--- Hidden Size: 32, Pooling: max, Optimizer: SGD, Epochs: 50 ---
Epoch 1/50, Loss: 1.4413, Val Loss: 1.4349, Accuracy: 3.82%
Epoch 2/50, Loss: 1.4347, Val Loss: 1.4283, Accuracy: 4.23%
Epoch 3/50, Loss: 1.4282, Val Loss: 1.4219, Accuracy: 4.47%
Epoch 4/50, Loss: 1.4218, Val Loss: 1.4156, Accuracy: 4.65%
Epoch 5/50, Loss: 1.4155, Val Loss: 1.4093, Accuracy: 4.97%
Epoch 6/50, Loss: 1.4093, Val Loss: 1.4032, Accuracy: 5.11%
Epoch 7/50, Loss: 1.4031, Val Loss: 1.3971, Accuracy: 5.39%
Epoch 8/50, Loss: 1.3970, Val Loss: 1.3910, Accuracy: 6.61%
Epoch 9/50, Loss: 1.3910, Val Loss: 1.3851, Accuracy: 7.77%
Epoch 10/50, Loss: 1.3851, Va

In [17]:
print("Best Hyperparameters:", best_params)
print("Best Accuracy:", best_accuracy)


Best Hyperparameters: {'hidden_size': 64, 'pooling_type': 'avg', 'optimizer': 'Adam', 'epochs': 350}
Best Accuracy: 0.9919127183947509
